In [1]:
import numpy as np
import pandas as pd 
data = pd.read_csv('preprocessed_emails_spam.csv', encoding='ISO-8859-1')

# data = np.genfromtxt('emails.csv', delimiter=',', skip_header=1)
# replace any NaN values in the 'email' column with an empty string
data['email'] = data['email'].fillna('')

# use the str.contains() method to find emails that contain 'Nan'
mask = data['email'].str.contains('Nan')

# filter out the emails that contain 'Nan'
data = data[~mask]




FileNotFoundError: [Errno 2] No such file or directory: 'preprocessed_emails_spam.csv'

In [3]:
data_ham_easy = pd.read_csv('preprocessed_emails_easy_ham.csv', encoding='ISO-8859-1')
data_ham_hard = pd.read_csv('preprocessed_emails_hard_ham.csv', encoding='ISO-8859-1')

print(f"shape of ham easy : {data_ham_easy.shape} and the shape of hard ham is {data_ham_hard.shape} ")

shape of ham easy : (2551, 2) and the shape of hard ham is (250, 2) 


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read in the CSV files
easy_ham = pd.read_csv('preprocessed_emails_easy_ham.csv')
hard_spam = pd.read_csv('preprocessed_emails_hard_ham.csv')
spam = pd.read_csv('preprocessed_emails_spam.csv')

# Combine the data frames
data = pd.concat([easy_ham, hard_spam, spam])
data = data.drop(columns=data.columns[data.columns.str.contains('Unnamed')])
data['Spam']=data['label'].apply(lambda x:1 if x=='spam' else 0)
data = data.drop(columns=data.columns[data.columns.str.contains('label')])
data= data.rename(columns={'email' : 'email2'})
# drop the "emails" column

# rename preprocessed_email to email 
data= data.rename(columns={'preprocessed_email' : 'email'})
# Replace NaN values in 'email' column with values from 'email2' column
data['email'] = np.where(pd.isnull(data['email']), data['email2'], data['email'])
data = data.drop("email2", axis=1)

# data =data.fillna("")
# replace any NaN values in the 'email' column with an empty string
data['email'] = data['email'].fillna('')


# use the str.contains() method to find emails that contain 'Nan'
mask = data['email'].str.contains('Nan')

# filter out the emails that contain 'Nan'
data = data[~mask]
data


,email,Spam
0,date wed nombr nombr aug nombr nombr nombr no...,0
1,martin a post tasso papadopoulo the greek scu...,0
2,man threaten explos in moscow thursday august...,0
3,klez the viru that wont die alreadi the most ...,0
4,on wed aug nombr nombr nombr nombr nombr nomb...,0
...,...,...
1391,medic transcript medic bill electrician inter...,1
1392,prefer nonsmok just what the doctor order cas...,1
1393,dear subscrib if i could show you a way to ge...,1
1394,midsumm custom appreci sale to express our ap...,1


In [5]:
data = data.sample(frac=1).reset_index(drop=True)
data

,email,Spam
0,a href nombr d httpaddr border nombr d nombr ...,1
1,skip ill tri a checkout into a new directori ...,0
2,opportun is knock whi becaus mortgag rate are...,1
3,blogstart dublin someth from the archiv daev ...,0
4,url httpaddr date nombr nombr nombr nombr nom...,0
...,...,...
4192,yannick gingra wrote i am wonder if there are...,0
4193,are you a littl behind the time not to worri ...,1
4194,tabl width nombr d nombr nombr nombr align no...,1
4195,im still see razor report bomb out with a sma...,0


In [6]:
from sklearn.model_selection import train_test_split
# X_train,X_test,y_train,y_test=train_test_split(data.email,data.Spam,test_size=0.2)

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score

import numpy as np

In [8]:
# Load a dataset
texts = data.email
labels = data.Spam

In [11]:

# Tokenize the texts
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
X = tokenizer.texts_to_sequences(data['email'].values)
X = pad_sequences(X)

# Convert label data to binary values
# y = pd.get_dummies(data['Spam']).values
print('Found %s unique tokens.' % len(word_index))
X

Found 46998 unique tokens.


array([[   0,    0,    0, ...,   46,   83,   12],
       [   0,    0,    0, ...,   10, 2760, 1295],
       [   0,    0,    0, ...,  221,   30,   12],
       ...,
       [   0,    0,    0, ...,   46,   83,   12],
       [   0,    0,    0, ...,   46, 1443,   12],
       [   0,    0,    0, ...,   10, 4175,   12]])

In [11]:
from sklearn.feature_extraction.text import CountVectorizer 
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [12]:

# # Convert the text data into a numerical representation using bag-of-words
# vectorizer = CountVectorizer(max_features=1000)
# X = vectorizer.fit_transform(data)

# # Encode the labels into numerical form
# le = LabelEncoder()
# y = le.fit_transform(labels)

# Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [13]:
# # Create an RNN model
# model = Sequential()
# model.add(Embedding(10000, 32, input_length=X.shape[1]))
# model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(1, activation='sigmoid'))

In [14]:
# Define the model architecture
from keras.layers import Bidirectional, Dropout


model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=32, input_length=X.shape[1]))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5)) 
model.add(Bidirectional(LSTM(32)))       
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense, Embedding, GlobalAveragePooling1D, Dropout, Attention
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # Define input sequence length and number of classes
# max_len = 128
# num_classes = 2

# # Define input layer
# inputs = Input(shape=(max_len,), dtype='int32')

# # Define embedding layer
# embedding = Embedding(input_dim=10000, output_dim=32, input_length=max_len)(inputs)

# # Define encoder layer with self-attention mechanism
# encoder = Attention(use_scale=True)([embedding, embedding, embedding])

# # Define classifier layers
# x = GlobalAveragePooling1D()(encoder)
# x = Dropout(0.5)(x)
# outputs = Dense(num_classes, activation='softmax')(x)

# # Define model architecture
# model = Model(inputs=inputs, outputs=outputs)

# # Compile model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
from keras.models import Model
from keras.layers import Input, Concatenate, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D
from keras.layers.recurrent import LSTM

# Define model 1
input1 = Input(shape=(X.shape[1],))
embedding1 = Embedding(input_dim=10000, output_dim=32, input_length=X.shape[1])(input1)
conv1 = Conv1D(filters=32, kernel_size=3, activation='relu')(embedding1)
pool1 = MaxPooling1D(pool_size=2)(conv1)
lstm1 = LSTM(64)(pool1)
dropout1 = Dropout(0.5)(lstm1)
output1 = Dense(1, activation='sigmoid')(dropout1)
model1 = Model(inputs=input1, outputs=output1)

# Define model 2
input2 = Input(shape=(X.shape[1],))
embedding2 = Embedding(input_dim=10000, output_dim=32, input_length=X.shape[1])(input2)
lstm2 = LSTM(128)(embedding2)
dropout2 = Dropout(0.5)(lstm2)
output2 = Dense(1, activation='sigmoid')(dropout2)
model2 = Model(inputs=input2, outputs=output2)

# Combine models
combined = Concatenate()([model1.output, model2.output])
ensemble_output = Dense(1, activation='sigmoid')(combined)
ensemble_model = Model(inputs=[model1.input, model2.input], outputs=ensemble_output)


In [15]:
# Compile the model

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
X_train

array([[   0,    0,    0, ...,    2, 8682,   29],
       [   0,    0,    0, ...,    1,  119,   12],
       [   0,    0,    0, ...,   46,  828,   12],
       ...,
       [   0,    0,    0, ...,   18,   17, 4069],
       [   0,    0,    0, ...,   60,   46,   12],
       [   0,    0,    0, ...,  569,   26,   65]])

In [14]:
from sklearn.preprocessing import MinMaxScaler
# Normalize data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train)
print(X_test.shape)


[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.00320513e-04
  8.73880221e-01 2.90522941e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00160256e-04
  1.19778561e-02 1.20216390e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 4.60737179e-03
  8.33417212e-02 1.20216390e-03]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.80288462e-03
  1.71112229e-03 4.07633741e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 6.00961538e-03
  4.63009562e-03 1.20216390e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 5.69911859e-02
  2.61701057e-03 6.51172110e-03]]
(840, 12972)


In [17]:

model.fit(X_train, y_train, epochs=10, batch_size=128)

Epoch 1/10


In [ ]:
# Make predictions on the testing set
y_pred = model.predict(X_test)
y_pred

In [ ]:
# Evaluate the accuracy of the model
accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)


27/27 [==============================] - 0s 13ms/step - loss: 0.1274 - acc: 0.9714
Accuracy: [0.12736640870571136, 0.9714285731315613]


In [ ]:
def predict(email):
    model.predict(email)

In [ ]:
# Save the model to disk
model.save('LSTM.h5')

In [ ]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), input_shape=X_train.shape[1:],padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

model.add(Conv2D(16, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))